<a href="https://colab.research.google.com/github/kafSaugat7/UnderGrad-Project/blob/main/Personal_Data_Vault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import base64
import getpass
import json
import os
from datetime import datetime

# --------- Class Definitions ---------

class DataItem:
    def __init__(self, title, content):
        self.title = title
        self.content = content
        self.created_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def to_dict(self):
        return {
            "title": self.title,
            "content": base64.b64encode(self.content.encode()).decode(),
            "created_date": self.created_date
        }

class Vault:
    def __init__(self, owner):
        self.owner = owner
        self.data_items = []

    def add_item(self, title, content):
        self.data_items.append(DataItem(title, content))

    def view_items(self):
        if not self.data_items:
            print("\n[!] Vault is empty.")
        else:
            for i, item in enumerate(self.data_items, 1):
                print(f"\n[{i}] {item.title} - {item.created_date}")
                print(f"    Content: {item.content}")

    def save_to_file(self, filename):
        data = [item.to_dict() for item in self.data_items]
        with open(filename, 'w') as f:
            json.dump(data, f)
        print("\n[+] Vault saved to", filename)

    def load_from_file(self, filename):
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                data = json.load(f)
                for entry in data:
                    decoded = base64.b64decode(entry['content']).decode()
                    item = DataItem(entry['title'], decoded)
                    item.created_date = entry['created_date']
                    self.data_items.append(item)
            print("[+] Loaded vault data.")

class User:
    def __init__(self, username, pin):
        self.username = username
        self.pin = pin
        self.vault = Vault(owner=username)

    def authenticate(self):
        entered = getpass.getpass("Enter PIN to unlock vault: ")
        return entered == self.pin

# --------- Main Program Logic ---------

def main():
    print("Welcome to Personal Data Vault")
    username = input("Enter your username: ")
    pin = getpass.getpass("Set your 4-digit PIN: ")

    user = User(username, pin)
    user.vault.load_from_file(f"{username}_vault.json")

    if not user.authenticate():
        print("[X] Incorrect PIN. Access Denied.")
        return

    while True:
        print("\n==== Vault Menu ====")
        print("1. Add new data")
        print("2. View stored items")
        print("3. Save and exit")
        choice = input("Enter choice: ")

        if choice == '1':
            title = input("Enter title: ")
            content = input("Enter secret content: ")
            user.vault.add_item(title, content)
            print("[+] Item added to vault.")

        elif choice == '2':
            user.vault.view_items()

        elif choice == '3':
            user.vault.save_to_file(f"{username}_vault.json")
            print("[✓] Exiting. Vault secured.")
            break

        else:
            print("[!] Invalid choice.")

if __name__ == "__main__":
    main()